In [1]:
import numpy as np
import datetime

In [2]:
np.random.seed(12345)    
def generateBinaryData(n, T, R_scales = 1):
    L = np.random.randint(low=0, high=2, size=(n,T))
    R = np.zeros((n,T))
    means = np.random.random_sample() * R_scales
    stds = np.random.random_sample()
    for i in range(T):
        R[:,i] = np.random.logistic(loc=means, scale=stds, size=(n,))
        R[:,i] = np.where(R[:,i] > 0, 1, 0)
        means = np.mean(R[:,:i+1] + L[:,:i+1])
        stds = np.std(R[:,:i+1] + L[:,:i+1])
    
    L_indicator = (R != 0)
    obsL = np.where(L_indicator, L, float('nan'))
    return obsL, R

In [3]:
def logisprob(X, W):
    return (1 / (1 + np.exp(-(np.dot(X,W)))))

def logisreg(X, y):
    w = np.zeros(np.shape(X)[1])
    while True:
        w_old = np.copy(w)
        p = np.zeros(np.shape(X)[0])
        for i in range(np.shape(X)[0]):
            p[i] = logisprob(X[i], w)
        W = np.diag(p * (1 - p))
        w = w + np.dot(np.dot(np.linalg.pinv(np.dot(np.dot(np.transpose(X), W), X)), np.transpose(X)), (y - p))
        if np.linalg.norm(w_old - w) < 0.001:
            break
    return np.transpose(w)

def linreg(X, y):
    ones = np.ones((X.shape[0],1))
    X = np.concatenate((ones, X), 1)
    A = np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(X), X)), np.transpose(X)), y)
    return A

In [4]:
def createWeightVecs(T, n, obsL, R):
    weight_vecs = []
    for t in range(T):
        priorObserved = []
        if t == 0:
            weight_vecs.append(np.isnan(obsL[:,t]).sum() / n)
        else:
            for i in range(n):
                if not np.isnan(obsL[i,:t]).any():
                    priorObserved.append(i)
            priorObserved = np.array(priorObserved)
            LR_arr = np.concatenate((obsL[priorObserved,:t], R[priorObserved,:t+1]), axis = 1)
            weight_vecs.append(logisreg(LR_arr[:,:-1], LR_arr[:,-1]))
    return weight_vecs

In [5]:
def createL_Weights(T, n, obsL):
    L_weights = []
    sigmas = []
    for t in range(T):
        priorObserved = []
        if t == 0:
            L_weights.append(np.nanmean(obsL[:,t]))
            sigmas.append(np.nanstd(obsL[:,t]))
        else:
            for i in range(n):
                if not np.isnan(obsL[i,:t+1]).any():
                    priorObserved.append(i)
            priorObserved = np.array(priorObserved)
            L_weights.append(linreg(obsL[priorObserved,:t], obsL[priorObserved,t]))
            ones = np.ones((obsL[priorObserved,t].shape[0], 1))
            X = np.concatenate((ones, obsL[priorObserved,:t]), 1)
            sigmas.append(np.sqrt(np.sum((obsL[priorObserved,t] - np.dot(X, L_weights[-1]))**2) / (X.shape[0] - t)))
    EY = np.nanmean(obsL[:,-1])
    return L_weights, sigmas, EY

In [31]:
def sampleE(row, EY, t, L_weights, sigmas, K, beta, perturbed_beta_plus, perturbed_beta_min):
    E = 0
    E_prime_p = 0
    E_prime_m = 0
    first_nan = float('inf')
    for j in range(row.shape[0] - 1):
        if np.isnan(row[j]):
            first_nan = j
            break
        else:
            pass
    first_nan = min(t+1, first_nan)
    orig_first_nan = first_nan
    if first_nan >= row.shape[0] - 1:
        E = np.dot(np.array([1] + row[:-1].tolist()), beta)
        E_prime_p = np.dot(np.array([1] + row[:-1].tolist()), perturbed_beta_plus)
        E_prime_m = np.dot(np.array([1] + row[:-1].tolist()), perturbed_beta_min)
        return E - EY, (E_prime_p - E_prime_m) - EY
    else:
        for k in range(K):
            if first_nan == 0:
                regress_cov = [1, L_weights[0] + np.random.normal(0, sigmas[0])]
                first_nan += 1
            else:
                regress_cov = [1] + row[:first_nan].tolist()
            for j in range(first_nan, row.shape[0] - 1):
                regress_cov.append(np.dot(np.array(regress_cov), L_weights[j]) + np.random.normal(0,sigmas[j]))
            sample = np.random.normal(0,sigmas[-1])
            E += np.dot(np.array(regress_cov), beta) + sample
            E_prime_p += np.dot(np.array(regress_cov), perturbed_beta_plus) + sample
            E_prime_m += np.dot(np.array(regress_cov), perturbed_beta_min) + sample
            first_nan = orig_first_nan
        return (E / K) - EY, ((E_prime_p - E_prime_m) / K) - EY

In [45]:
def main():
    L_output = []
    EY_output = []
    for l in range(100):
        print('Experiment Num', l+1)
        n = 10000
        T = 3
        R_scales = .0625
        obsL, R = generateBinaryData(n,T, R_scales)
        weight_vecs = createWeightVecs(T, n, obsL, R)
        L_weights, sigmas, EY = createL_Weights(T, n, obsL)
        new_beta = np.zeros(T)
        epsilon = .0005
        iteration_num = 1
        print(L_weights)

        while True:
            print(iteration_num)
            iteration_num += 1
            G = np.zeros(T)
            E = 0
            perturb_val = 1/100

            for i in range(n): #iterate over rows and sum
                L2_fraction = 0
                for t in range(T-1):
                    if t == 0:
                        if np.isnan(obsL[i,t]):
                            pass
                        else:
                            R_observed = (t+1) * [1]
                            numerator = (1 - R[i,t]) - weight_vecs[t]
                            denominator = logisprob(np.array(obsL[i,:t+1].tolist() + R_observed), weight_vecs[t+1])
                            L2_fraction += numerator / denominator
                    else:
                        if np.isnan(obsL[i,:t+1]).any():
                            pass
                        else:
                            R_observed = t * [1]
                            numerator = R[i,t-1] * ((1 - R[i,t]) - logisprob(np.array(obsL[i,:t].tolist() + R_observed), weight_vecs[t]))
                            R_observed = (t+1) * [1]
                            denominator = logisprob(np.array(obsL[i,:t+1].tolist() + R_observed), weight_vecs[t+1])
                            L2_fraction += numerator / denominator
                    for j in range(T):
                        perturbed_beta_plus = np.copy(L_weights[-1])
                        perturbed_beta_minus = np.copy(L_weights[-1])
                        perturbed_beta_plus[j] +=  perturb_val
                        perturbed_beta_minus[j] -= perturb_val
                        Exp, E_prime = sampleE(obsL[i,:], EY, t, L_weights, sigmas, 5, L_weights[-1], perturbed_beta_plus, perturbed_beta_minus)
                        G[j] += (L2_fraction * E_prime) / (2 * perturb_val)
                        E += Exp / T
            new_beta = L_weights[-1] - (E / G)
            if np.linalg.norm(L_weights[-1] - new_beta) < epsilon:
                print(np.linalg.norm(L_weights[-1] - new_beta))
                print(new_beta)
                L_output.append(new_beta)
                EY_output.append(EY)
                break
            else:
                print('Difference:', np.linalg.norm(L_weights[-1] - new_beta))
                L_weights[-1] = np.copy(new_beta)
                print(L_weights[-1])
    np.save('./fixed_L_output_IF_' + str(R_scales), np.array(L_output))
    np.save('./fixed_EY_output_IF_' + str(R_scales), np.array(EY_output))
    print('Done!')

In [46]:
np.random.seed(12345)
main()

Experiment Num 1
[0.50528582615505091, array([ 0.50561249, -0.00704243]), array([ 0.47805648,  0.01096463,  0.02507111])]
1
0.000170750296683
[ 0.47795654  0.01086673  0.02497322]
Experiment Num 2
[0.50375093773443358, array([ 0.50509259, -0.01451788]), array([ 0.49543092,  0.01275502,  0.00191577])]
1
7.10383534204e-05
[ 0.4954725   0.01279576  0.0019565 ]
Experiment Num 3
[0.50197238658777121, array([ 0.52618698, -0.01470676]), array([ 0.48994913,  0.01331987,  0.01645389])]
1
0.000468903305325
[ 0.48967477  0.01305109  0.0161849 ]
Experiment Num 4
[0.49599847560975607, array([ 0.50637093, -0.02932408]), array([ 0.50273686, -0.01820053, -0.00097606])]
1
Difference: 0.00121635714977
[  5.03448907e-01  -1.75031454e-02  -2.78792909e-04]
2
Difference: 0.000697199998486
[  5.03857039e-01  -1.71034155e-02   1.20872024e-04]
3
0.000463568758747
[  5.04128408e-01  -1.68376330e-02   3.86605739e-04]
Experiment Num 5
[0.50261149055845722, array([ 0.50152439, -0.01202439]), array([ 0.50041336,  0

Difference: 0.000644810968912
[ 0.48147649 -0.0013371   0.02993561]
6
0.000435134321064
[ 0.48173121 -0.00108759  0.03018503]
Experiment Num 29
[0.50584969264326785, array([ 0.49559687,  0.01212514]), array([ 0.47314891,  0.01705364,  0.00491682])]
1
0.000352915263896
[ 0.47294227  0.01685135  0.00471452]
Experiment Num 30
[0.49812734082397003, array([ 0.49289564, -0.00237546]), array([ 0.48842568,  0.02519697,  0.01683285])]
1
0.000375746052781
[ 0.48820577  0.02498149  0.01661745]
Experiment Num 31
[0.50222222222222224, array([ 0.50279107, -0.00258986]), array([ 0.49042061,  0.00620963,  0.02058541])]
1
0.000398591578917
[ 0.49018732  0.00598109  0.02035689]
Experiment Num 32
[0.49380085238279736, array([ 0.48896195,  0.02505674]), array([ 0.48742432,  0.00166848,  0.01634106])]
1
2.00646297818e-05
[ 0.48741257  0.00165698  0.01632956]
Experiment Num 33
[0.51093439363817095, array([ 0.47719118,  0.00649123]), array([ 0.49145021,  0.03032335, -0.01749272])]
1
0.000177353051663
[ 0.491

0.000237977568969
[ 0.49005005  0.0145566   0.00156811]
Experiment Num 56
[0.50531705395825133, array([ 0.50442913,  0.00815557]), array([ 0.49715702,  0.02274813, -0.02719162])]
1
0.000272032813647
[ 0.49731621  0.02290411 -0.02703564]
Experiment Num 57
[0.50564414979394379, array([ 0.48430493,  0.01266214]), array([ 0.49923267,  0.00599196,  0.00377517])]
1
0.000290939298892
[ 0.49906239  0.0058251   0.00360841]
Experiment Num 58
[0.4995032783628055, array([ 0.50547809, -0.01021152]), array([ 0.53000119, -0.01617951, -0.03478295])]
1
0.000414174680355
[ 0.53024357 -0.01594204 -0.03454546]
Experiment Num 59
[0.49775959477888176, array([ 0.49593107, -0.00293928]), array([ 0.51265921, -0.01027458, -0.01188463])]
1
0.00026630264773
[ 0.51250333 -0.01042726 -0.01203731]
Experiment Num 60
[0.49793347766187757, array([ 0.5026764 ,  0.02115652]), array([ 0.51344204, -0.01132233, -0.02015745])]
1
0.000195765729135
[ 0.5135566  -0.01121011 -0.02004516]
Experiment Num 61
[0.49817251461988304, a

[0.49627720504009165, array([ 0.51305221, -0.01149888]), array([ 0.49106   ,  0.02770241, -0.00258165])]
1
0.000164384175173
[ 0.49115621  0.02779664 -0.00248738]
Experiment Num 84
[0.50086655112651646, array([ 0.49000951,  0.00999049]), array([ 0.49872697, -0.00252263, -0.00256646])]
1
Difference: 0.00103324049901
[ 0.49933167 -0.00193015 -0.00197409]
2
Difference: 0.000809589531158
[ 0.49980547 -0.00146591 -0.00150995]
3
Difference: 0.000713073076481
[ 0.50022281 -0.00105701 -0.00110117]
4
0.000463620243423
[ 0.50049414 -0.00079115 -0.00083538]
Experiment Num 85
[0.49641291351135913, array([ 0.50942484, -0.01120059]), array([ 0.49529364, -0.00125376,  0.01333563])]
1
0.000136816814179
[ 0.49521357 -0.00133218  0.01325717]
Experiment Num 86
[0.50181887803944092, array([ 0.49315715,  0.0032629 ]), array([ 0.50547399, -0.00480827, -0.02034114])]
1
Difference: 0.000513199935863
[ 0.50577442 -0.00451402 -0.02004698]
2
0.000404283260669
[ 0.50601109 -0.00428221 -0.01981526]
Experiment Num 

In [56]:
fnames = [['./fixed_L_output_IF_100.npy', './fixed_EY_output_IF_100.npy', 100], ['./fixed_L_output_IF_1000.npy', './fixed_EY_output_IF_1000.npy', 1000]]
fnames.append(['./fixed_L_output_IF_10000.npy', './fixed_EY_output_IF_10000.npy', 10000])
mean_error = []
for trio in fnames:
    mean_error.append(list())
    L_weights = np.load(trio[0])
    EY = np.load(trio[1])
    np.random.seed(12345)
    for k in range(100):
        n = trio[2]
        T = 3
        obsL, R = generateBinaryData(n,T)
        allObs = []
        for i in range(obsL.shape[0]):
            if not np.isnan(obsL[i,:-1]).any():
                allObs.append(i)
        ones = np.ones((obsL[np.array(allObs)].shape[0], 1))
        X = np.concatenate((ones, obsL[np.array(allObs),:-1]), 1)
        residuals = np.dot(X, L_weights[k]) - EY[k]
        mean_error[-1].append(np.mean(residuals))
    print('mean', np.mean(mean_error[-1]))
    print('var', np.var(mean_error[-1]))

mean 0.00146802182164
var 0.000312111183335
mean -8.97970945936e-05
var 1.45522579977e-05
mean 0.000119144483909
var 5.51341072892e-06


In [55]:
fnames = ['./fixed_L_output_IF_0.0625.npy', './fixed_L_output_IF_0.125.npy', './fixed_L_output_IF_0.25.npy']
fnames.append('./fixed_L_output_IF_0.5.npy')
fnames.append('./fixed_L_output_IF_10000.npy')
fnames.append('./fixed_L_output_IF_2.npy')
R_norms = []
true_param = [.5,0,0]
for fname in fnames:
    R_norms.append(list())
    L_weights = np.load(fname)
    for k in range(100):
        R_norms[-1].append(np.linalg.norm(L_weights[k] - np.array(true_param)))
    print('mean', np.mean(R_norms[-1]))
    print('var', np.var(R_norms[-1]))

mean 0.0221723395234
var 0.000112680124769
mean 0.0228696491185
var 0.000139700507137
mean 0.0218113167772
var 0.000134719675181
mean 0.0220815596462
var 0.000119196455397
mean 0.0187792384886
var 7.95070432194e-05
mean 0.0184245153455
var 0.000101570425161
